In [100]:
import logging
import feedgen
from feedgen.feed import FeedGenerator

from requests_html import HTMLSession

from IPython.display import display, Image

In [5]:
BASE_URL = 'https://www.maz-online.de/Lokales/Potsdam-Mittelmark/Werder-Havel'

In [6]:
sess = HTMLSession()
r = sess.get(BASE_URL)

In [106]:
def get_maz_articles(session, url):
    r = session.get(url)
    articles = []
    for pub_type in ('articles4', 'teaser3'):
        for article in r.html.find(f'div.pdb-{pub_type}-teaser'):
            try:
                link_elem = article.find(f'a.pdb-{pub_type}-teaser-breadcrumb-headline-title-link', first=True)
                link_abs_url = requests_html.urljoin(BASE_URL, link_elem.attrs['href'])

                pub_date_elem = article.find(f'span.pdb-{pub_type}-teaser-release-publicationdate', first=True)
                pub_date = ''
                if pub_date_elem:
                    pub_date = pub_date_elem.text

                headline = article.find(f'a.pdb-{pub_type}-teaser-breadcrumb-headline-title-link', first=True).text
                img = article.find('img', first=True)
                intro = article.find('p', first=True).text

                if pub_date:  # articles w/out a publication date are spam/listicles
                    article = {
                        'url': link_abs_url,
                        'pub_date': pub_date,
                        'headline': headline,
                        'intro': intro,
                        'image_url': img.attrs['src'],
                        'alt_text': img.attrs['alt']}
                    articles.append(article)
            except Exception as ex:
                logging.exception('Caught exception when parsing MAZ/Werder page.')
    return articles

In [107]:
articles = get_maz_articles(sess, BASE_URL)

In [136]:
def create_feed(articles, output_filepath=None):
    feed = FeedGenerator()
#     feed.load_extension('podcast', atom=True)
    feed.title(u"MAZ: Werder/Havel")
    feed.id(BASE_URL)
    feed.link(href='https://www.maz-online.de/', rel='self')
    feed.description(u"Inoffizieller RSS-Feed für MAZ Werder/Havel")
    feed.language('de')
    
    for article in articles:
        entry = feed.add_entry()
        entry.link(href=article['url'], rel='alternate')
        entry.title(article['headline'])
        entry.content(f"""<img src="{article['image_url']}" align="left" hspace="10"> {article["pub_date"]} {article["intro"]}""")
        entry.guid()
        
        
    if output_filepath:
        feed.rss_file(output_filepath)
    else:
        return feed.rss_str(pretty=True)

In [137]:
create_feed(articles, output_filepath='/tmp/maz.xml')

In [101]:
f = FeedGenerator()

In [103]:
fe = f.add_entry()

In [104]:
# hspace="50" vspace="50"